In [85]:
import openml
from openml import datasets, tasks, runs, flows, config, study
from openml.datasets import edit_dataset, fork_dataset, get_dataset
from openml.tasks import TaskType
import os, pandas as pd, sklearn, arff, pprint, numpy as np, seaborn as sns
from sklearn import neighbors
from tqdm import tqdm

config.apikey = '2474cbc06507fbf9677530945039be25'
config.server = 'https://www.openml.org/api/v1' # https://test.openml.org/api/v1/xml
config.set_cache_directory(os.path.expanduser('~/openml/cache'))
config.cachedir = '~/openml/cache'

# SCARF benchmark 수집
- 72개 데이터 중에서
    - mnist_784, Fashion-MNIST, CIFAR_10 제외 (69개)
    - cat 변수 없는거 제외 (23개)


In [86]:
# benchmark suite 중에서 우리가 원하는 'OpenML-CC18' 가져오기

benchmark_suite = openml.study.get_suite('OpenML-CC18') # obtain the benchmark suite
benchmark_suite

OpenML Benchmark Suite
ID..............: 99
Name............: OpenML-CC18 Curated Classification benchmark
Status..........: active
Main Entity Type: task
Study URL.......: https://www.openml.org/s/99
# of Data.......: 72
# of Tasks......: 72
Creator.........: https://www.openml.org/u/1
Upload Time.....: 2019-02-21 18:47:13

In [87]:
# benchmark suite에 존재하는 task ids, data ids

print(benchmark_suite.tasks) # 72개
print(benchmark_suite.data) # 72개

[3, 6, 11, 12, 14, 15, 16, 18, 22, 23, 28, 29, 31, 32, 37, 43, 45, 49, 53, 219, 2074, 2079, 3021, 3022, 3481, 3549, 3560, 3573, 3902, 3903, 3904, 3913, 3917, 3918, 7592, 9910, 9946, 9952, 9957, 9960, 9964, 9971, 9976, 9977, 9978, 9981, 9985, 10093, 10101, 14952, 14954, 14965, 14969, 14970, 125920, 125922, 146195, 146800, 146817, 146819, 146820, 146821, 146822, 146824, 146825, 167119, 167120, 167121, 167124, 167125, 167140, 167141]
[3, 6, 11, 12, 14, 15, 16, 18, 22, 23, 28, 29, 31, 32, 37, 44, 46, 50, 54, 151, 182, 188, 38, 307, 300, 458, 469, 554, 1049, 1050, 1053, 1063, 1067, 1068, 1590, 4134, 1510, 1489, 1494, 1497, 1501, 1480, 1485, 1486, 1487, 1468, 1475, 1462, 1464, 4534, 6332, 1461, 4538, 1478, 23381, 40499, 40668, 40966, 40982, 40994, 40983, 40975, 40984, 40979, 40996, 41027, 23517, 40923, 40927, 40978, 40670, 40701]


In [88]:
# dataset 가져오기

dlist = datasets.list_datasets(data_id=benchmark_suite.data, output_format='dataframe')
dlist

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
11,11,balance-scale,1,1,active,ARFF,288.0,3.0,49.0,3.0,5.0,625.0,0.0,0.0,4.0,1.0
12,12,mfeat-factors,1,1,active,ARFF,200.0,10.0,200.0,10.0,217.0,2000.0,0.0,0.0,216.0,1.0
14,14,mfeat-fourier,1,1,active,ARFF,200.0,10.0,200.0,10.0,77.0,2000.0,0.0,0.0,76.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40983,40983,wilt,2,4265,active,ARFF,4578.0,2.0,261.0,2.0,6.0,4839.0,0.0,0.0,5.0,1.0
40984,40984,segment,3,4265,active,ARFF,330.0,7.0,330.0,7.0,20.0,2310.0,0.0,0.0,19.0,1.0
40994,40994,climate-model-simulation-crashes,4,4265,active,ARFF,494.0,2.0,46.0,2.0,21.0,540.0,0.0,0.0,20.0,1.0
40996,40996,Fashion-MNIST,1,2506,active,ARFF,7000.0,10.0,7000.0,10.0,785.0,70000.0,0.0,0.0,784.0,1.0


In [89]:
# unique values of NumberOfInstancesWithMissingValues

dlist['NumberOfInstancesWithMissingValues'].unique()

array([   0.,   16.,   37., 3772.,   95.,    5., 3620.,  263.,  401.,
        528.])

In [147]:
# tasks 가져오기

tlist = openml.tasks.list_tasks(tag="OpenML-CC18", output_format="dataframe")
tlist

,tid,ttid,did,name,task_type,status,estimation_procedure,source_data,target_feature,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
3,3,TaskType.SUPERVISED_CLASSIFICATION,3,kr-vs-kp,Supervised Classification,active,10-fold Crossvalidation,3,class,1669,3.0,1527,2,37,3196,0,0,0,37
6,6,TaskType.SUPERVISED_CLASSIFICATION,6,letter,Supervised Classification,active,10-fold Crossvalidation,6,class,813,26.0,734,26,17,20000,0,0,16,1
11,11,TaskType.SUPERVISED_CLASSIFICATION,11,balance-scale,Supervised Classification,active,10-fold Crossvalidation,11,class,288,3.0,49,3,5,625,0,0,4,1
12,12,TaskType.SUPERVISED_CLASSIFICATION,12,mfeat-factors,Supervised Classification,active,10-fold Crossvalidation,12,class,200,10.0,200,10,217,2000,0,0,216,1
14,14,TaskType.SUPERVISED_CLASSIFICATION,14,mfeat-fourier,Supervised Classification,active,10-fold Crossvalidation,14,class,200,10.0,200,10,77,2000,0,0,76,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167121,167121,TaskType.SUPERVISED_CLASSIFICATION,40923,Devnagari-Script,Supervised Classification,active,10-fold Crossvalidation,40923,character,2000,46.0,2000,46,1025,92000,0,0,1024,1
167124,167124,TaskType.SUPERVISED_CLASSIFICATION,40927,CIFAR_10,Supervised Classification,active,10-fold Crossvalidation,40927,class,6000,10.0,6000,10,3073,60000,0,0,3072,1
167125,167125,TaskType.SUPERVISED_CLASSIFICATION,40978,Internet-Advertisements,Supervised Classification,active,10-fold Crossvalidation,40978,class,2820,2.0,459,2,1559,3279,0,0,3,1556
167140,167140,TaskType.SUPERVISED_CLASSIFICATION,40670,dna,Supervised Classification,active,10-fold Crossvalidation,40670,class,1654,3.0,765,3,181,3186,0,0,0,181


In [91]:
# get non-intersection between two lists

list_1 = tlist['did'].unique()
list_2 = benchmark_suite.data

len(list(set(list_1) - set(list_2)))

0

In [92]:
# get non-intersection between two lists

list_1 = tlist['source_data'].astype(int).unique()
list_2 = benchmark_suite.data

len(list(set(list_1) - set(list_2)))

0

In [93]:
# unique values of NumberOfInstancesWithMissingValues

tlist['NumberOfInstancesWithMissingValues'].unique()

array([   0,   16,   37,   95, 3772,    5, 3620,  263,  401,  528])

In [94]:
# unique values of task_type

tlist['task_type'].unique()

array(['Supervised Classification'], dtype=object)

In [95]:
# 여기서부터 데이터 수집

In [114]:
print(os.getcwd())
data_path = '/home/ubuntu/User/LEE/OpenML-dataset/data/'
# os.makedirs(data_path)

In [129]:
dataset_dict = {}

for did in tqdm(dlist['did']):
    # get dataset name from dlist
    name = dlist.loc[dlist['did']==did, 'name'].values[0]
    
    # get dataset from dlist
    odata = datasets.get_dataset(did)
    X, y, categorical_indicator, attribute_names = odata.get_data(
                                                            target=odata.default_target_attribute, 
                                                            include_row_id=True,
                                                            include_ignore_attribute=True,
    )
    y.name = 'class' # y변수 이름 통일
    
    if True in categorical_indicator: # cat변수 있는 데이터만
        
        # get num, cat feature names
        num_feat_names = [attribute_names[i] for i in range(len(attribute_names)) if categorical_indicator[i]==False]
        cat_feat_names = [attribute_names[i] for i in range(len(attribute_names)) if categorical_indicator[i]==True]

        # sort features names by num_feat_names and cat_feat_names
        X = X[num_feat_names+cat_feat_names] 
        
        # get split from tlist
        task = openml.tasks.get_task(tlist[tlist['did']==did]['tid'].values[0])
        train_indices, test_indices = task.get_train_test_split_indices(repeat=0, fold=0, sample=0,)
        X_train, X_test, y_train, y_test = X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]
        train = pd.concat([X_train, y_train], axis=1)
        test = pd.concat([X_test, y_test], axis=1)
        
        # create a folder path
        os.makedirs(data_path+name, exist_ok=True)
        # save train and test data
        train.to_csv(data_path+name+'/train.csv', index=False)
        test.to_csv(data_path+name+'/test.csv', index=False)
        
        # save cat feat names
        pd.DataFrame(cat_feat_names).to_csv(data_path+name+'/cat_feat_names.csv', index=False)


100%|██████████| 72/72 [00:19<00:00,  3.73it/s]


# 추가논문 benchmark 수집
- Why do tree-based models still outperform deep learning on tabular data
- Categorical classification, OpenML benchmark (7개)
    - https://www.openml.org/search?type=benchmark&sort=tasks_included&study_type=task&id=300
    

In [131]:
benchmark_suite = openml.study.get_suite(300) # obtain the benchmark suite
benchmark_suite

OpenML Benchmark Suite
ID..............: 300
Name............: Tabular benchmark categorical classification
Status..........: in_preparation
Main Entity Type: task
Study URL.......: https://www.openml.org/s/300
# of Data.......: 7
# of Tasks......: 7
Creator.........: https://www.openml.org/u/26324
Upload Time.....: 2022-07-12 18:58:05

In [132]:
# benchmark suite에 존재하는 task ids, data ids

print(benchmark_suite.tasks) # 
print(benchmark_suite.data) #

[361110, 361111, 361112, 361113, 361114, 361115, 361116]
[44156, 44157, 44158, 44159, 44160, 44161, 44162]


In [133]:
# dataset 가져오기

dlist = datasets.list_datasets(data_id=benchmark_suite.data, output_format='dataframe')
dlist

,did,name,version,uploader,status,format,MajorityClassSize,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
44156,44156,electricity,13,26324,active,arff,19237.0,19237.0,2.0,9.0,38474.0,0.0,0.0,7.0,2.0
44157,44157,eye_movements,8,26324,active,arff,3804.0,3804.0,2.0,24.0,7608.0,0.0,0.0,20.0,4.0
44158,44158,KDDCup09_upselling,4,26324,active,arff,2516.0,2516.0,2.0,46.0,5032.0,0.0,0.0,34.0,12.0
44159,44159,covertype,13,26324,active,arff,211840.0,211840.0,2.0,55.0,423680.0,0.0,0.0,10.0,45.0
44160,44160,rl,4,26324,active,arff,2485.0,2485.0,2.0,13.0,4970.0,0.0,0.0,5.0,8.0
44161,44161,road-safety,6,26324,active,arff,55881.0,55881.0,2.0,33.0,111762.0,0.0,0.0,29.0,4.0
44162,44162,compass,3,26324,active,arff,8322.0,8322.0,2.0,18.0,16644.0,0.0,0.0,8.0,10.0


In [134]:
# unique values of NumberOfInstancesWithMissingValues

dlist['NumberOfInstancesWithMissingValues'].unique()

array([0.])

In [144]:
benchmark_suite

OpenML Benchmark Suite
ID..............: 300
Name............: Tabular benchmark categorical classification
Status..........: in_preparation
Main Entity Type: task
Study URL.......: https://www.openml.org/s/300
# of Data.......: 7
# of Tasks......: 7
Creator.........: https://www.openml.org/u/26324
Upload Time.....: 2022-07-12 18:58:05

In [152]:
# tasks 가져오기

tlist = openml.tasks.get_tasks(benchmark_suite.tasks)
tlist

[OpenML Classification Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
 Task ID..............: 361110
 Task URL.............: https://www.openml.org/t/361110
 Estimation Procedure.: crossvalidation
 Evaluation Measure...: predictive_accuracy
 Target Feature.......: class
 # of Classes.........: 2
 Cost Matrix..........: Available,
 OpenML Classification Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
 Task ID..............: 361111
 Task URL.............: https://www.openml.org/t/361111
 Estimation Procedure.: crossvalidation
 Evaluation Measure...: predictive_accuracy
 Target Feature.......: label
 # of Classes.........: 2
 Cost Matrix..........: Available,
 OpenML Classification Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
 Task ID..............: 361112
 Task URL.............: https://www.openml.org/t/361112
 Estimation Procedure.: crossvalidation
 Evaluatio

In [158]:
# get non-intersection between two lists

list_1 = [i.dataset_id for i in tlist]
list_2 = benchmark_suite.data

len(list(set(list_1) - set(list_2)))

0

In [140]:
# 여기서부터 데이터 수집

In [161]:
print(os.getcwd())
data_path = '/home/ubuntu/User/LEE/OpenML-dataset/data_2/'
# os.makedirs(data_path)

/home/ubuntu/User/LEE/OpenML-dataset


In [163]:
tlist[0].dataset_id

44156

In [170]:
dataset_dict = {}

for did in tqdm(dlist['did']):
    # get dataset name from dlist
    name = dlist.loc[dlist['did']==did, 'name'].values[0]
    
    # get dataset from dlist
    odata = datasets.get_dataset(did)
    X, y, categorical_indicator, attribute_names = odata.get_data(
                                                            target=odata.default_target_attribute, 
                                                            include_row_id=True,
                                                            include_ignore_attribute=True,
    )
    y.name = 'class' # y변수 이름 통일
    
    if True in categorical_indicator: # cat변수 있는 데이터만
        
        # get num, cat feature names
        num_feat_names = [attribute_names[i] for i in range(len(attribute_names)) if categorical_indicator[i]==False]
        cat_feat_names = [attribute_names[i] for i in range(len(attribute_names)) if categorical_indicator[i]==True]

        # sort features names by num_feat_names and cat_feat_names
        X = X[num_feat_names+cat_feat_names] 
        
        # get split from tlist
        task = [i for i in tlist if i.dataset_id==did][0]
        train_indices, test_indices = task.get_train_test_split_indices(repeat=0, fold=0, sample=0,)
        X_train, X_test, y_train, y_test = X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]
        train = pd.concat([X_train, y_train], axis=1)
        test = pd.concat([X_test, y_test], axis=1)
        
        # create a folder path
        os.makedirs(data_path+name, exist_ok=True)
        # save train and test data
        train.to_csv(data_path+name+'/train.csv', index=False)
        test.to_csv(data_path+name+'/test.csv', index=False)
        
        # save cat feat names
        pd.DataFrame(cat_feat_names).to_csv(data_path+name+'/cat_feat_names.csv', index=False)


100%|██████████| 7/7 [01:04<00:00,  9.22s/it]


# public study 찾기 
- self-supervised
- supervised